### Importación de librerías

In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Definición de variables

In [2]:
DATASET_PATH = "../../dataset/gestos_cara"
MODEL_FILE = "modelo_gestos_cara.pkl"

### Lógica de entreno

In [4]:
# Configuración de MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh

# static_image_mode=True: Optimizado para procesar fotos sueltas (más preciso)
# refine_landmarks=True: Añade puntos del iris (clave para detectar ojos cerrados)
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

data = []
labels = []

# Mapeo de carpetas a números (Debe coincidir con tus carpetas)
mapa_etiquetas = {
    "0_Neutro": 0,
    "1_Ojos_Cerrados": 1,
    "2_Cabeza_Der": 2,
    "3_Cabeza_Izq": 3
}

print(f"--- ENTRENADOR FACIAL ---")
print(f"Buscando imágenes en: {DATASET_PATH}...")

# 1. BUCLE DE LECTURA DE IMÁGENES
for nombre_carpeta, etiqueta_num in mapa_etiquetas.items():
    ruta_carpeta = os.path.join(DATASET_PATH, nombre_carpeta)
    
    if not os.path.exists(ruta_carpeta):
        print(f"AVISO: No existe la carpeta {nombre_carpeta}. Saltando...")
        continue
        
    archivos = os.listdir(ruta_carpeta)
    print(f"Procesando '{nombre_carpeta}': {len(archivos)} fotos found.")
    
    count_ok = 0
    count_fail = 0
    
    for archivo in archivos:
        # Filtrar solo imágenes
        if not (archivo.endswith(".jpg") or archivo.endswith(".png")):
            continue
            
        ruta_img = os.path.join(ruta_carpeta, archivo)
        img = cv2.imread(ruta_img)
        
        if img is None:
            continue

        # MediaPipe necesita RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Procesar la cara
        results = face_mesh.process(img_rgb)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Extraemos los 478 puntos (x, y)
                landmark_list = []
                for lm in face_landmarks.landmark:
                    landmark_list.append(lm.x)
                    landmark_list.append(lm.y)
                    # No usamos Z por ahora para simplificar, X e Y bastan para gestos
                
                data.append(landmark_list)
                labels.append(etiqueta_num)
                count_ok += 1
        else:
            count_fail += 1

    print(f"  -> {count_ok} caras detectadas | {count_fail} fallos/sin cara")

# 2. ENTRENAMIENTO
if len(data) > 0:
    print("\n--- ENTRENANDO INTELIGENCIA ARTIFICIAL ---")
    
    # Convertir listas a arrays de Numpy (más rápido)
    data = np.array(data)
    labels = np.array(labels)

    # Separar 20% de los datos para prueba (Test) y 80% para entrenar (Train)
    # Esto nos dice qué tan bueno es el modelo antes de guardarlo
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

    model = RandomForestClassifier()
    
    # Entrenar con el 80%
    model.fit(x_train, y_train)

    # Probar con el 20% restante
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Precisión estimada del modelo: {accuracy * 100:.2f}%")
    
    if accuracy < 0.7:
        print("ADVERTENCIA: La precisión es baja. Intenta tomar mejores fotos o más cantidad.")

    # Re-entrenar con EL 100% DE LOS DATOS para el archivo final
    print("Generando modelo final con todos los datos...")
    model.fit(data, labels)
    
    with open(MODEL_FILE, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"¡LISTO! Modelo guardado como: '{MODEL_FILE}'")
    print("Ya puedes usar este archivo en tu juego.")

else:
    print("\nERROR CRÍTICO: No se encontraron datos válidos.")
    print("Asegúrate de haber ejecutado 'recolector_cara.py' primero y tener fotos en las carpetas.")


--- ENTRENADOR FACIAL ---
Buscando imágenes en: ../../dataset/gestos_cara...
Procesando '0_Neutro': 330 fotos found.


c:\Users\lucia\anaconda33\envs\memo_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


  -> 330 caras detectadas | 0 fallos/sin cara
Procesando '1_Ojos_Cerrados': 334 fotos found.
  -> 329 caras detectadas | 5 fallos/sin cara
Procesando '2_Cabeza_Der': 300 fotos found.
  -> 198 caras detectadas | 102 fallos/sin cara
Procesando '3_Cabeza_Izq': 300 fotos found.
  -> 159 caras detectadas | 141 fallos/sin cara

--- ENTRENANDO INTELIGENCIA ARTIFICIAL ---
Precisión estimada del modelo: 100.00%
Generando modelo final con todos los datos...
¡LISTO! Modelo guardado como: 'modelo_gestos_cara.pkl'
Ya puedes usar este archivo en tu juego.
